## Install prerequisites

In [ ]:
# !pip install tabulate json

In [ ]:
# Use this command to install the current directory as a python package in editable mode.
# Run the command from the same directory as the pyproject.toml file.
# pip install -e .

In [67]:
# import stuff
import spynns
import tabulate
import json
import random
import sys
from itertools import pairwise
import math
# from IPython.core.debugger import Pdb
from spynns.tbp_node import PlasticNode

In [68]:
seed = random.randint(0, sys.maxsize)
rng = random.Random(seed)
print(seed)

def make_layer(net, n, **kwargs):
    return [PlasticNode(net, **kwargs) for _ in range(n)]

def layer_rand(net, n, **kwargs):
    return [PlasticNode(net,
        threshold=rng.uniform(0, 2),
        **kwargs,
    ) for _ in range(n)]

def connectw(parent, child, **kwargs):
    parent.add_output(
        child,
        weight=rng.uniform(0, 2),
        **kwargs
    )

def connectw_multiple(parent, children, **kwargs):
    for child in children:
        connectw(parent, child, **kwargs)

def fully_connectw_layers(parents, children, **kwargs):
    for parent in parents:
        connectw_multiple(parent, children, **kwargs)


4082953305450607197


In [106]:
net = spynns.Network()

inputs = make_layer(net, 2, threshold=0.1, refractory=0.01)
outputs = make_layer(net, 5, threshold=1.0, refractory=0.01)

h = [layer_rand(net, 5) for i in range(2)]

for a, b in pairwise([inputs, *h, outputs]):
    fully_connectw_layers(a, b, delay=0.02)

# connect the output of the first node to the second node
# inputs[0].add_output(outputs[0], weight=100, delay=1.0)
# connect the output of the second node to the first node, creating a loop
# outputs[0].add_output(inputs[0], weight=11, delay=0.2)

nodes = inputs + outputs
[nodes.extend(layer) for layer in h]
nodes

[Node at 234d123bb30 w/ Threshold: 0.1, Leak: None, refract: 0.01, connections: ['<Edge w: 0.8591449323988125, d: 0.02, dest: 3bf0>', '<Edge w: 1.6497769066717631, d: 0.02, dest: 12e0>', '<Edge w: 0.2663139461433657, d: 0.02, dest: df70>', '<Edge w: 1.1800750516067615, d: 0.02, dest: d610>', '<Edge w: 1.0893540715799113, d: 0.02, dest: d910>'],
 Node at 234d1260f50 w/ Threshold: 0.1, Leak: None, refract: 0.01, connections: ['<Edge w: 0.9685357986258842, d: 0.02, dest: 3bf0>', '<Edge w: 0.6598215096011175, d: 0.02, dest: 12e0>', '<Edge w: 1.787766528736834, d: 0.02, dest: df70>', '<Edge w: 0.4827945445039117, d: 0.02, dest: d610>', '<Edge w: 1.5379202424062188, d: 0.02, dest: d910>'],
 Node at 234d1263380 w/ Threshold: 1.0, Leak: None, refract: 0.01, connections: [],
 Node at 234d1263ce0 w/ Threshold: 1.0, Leak: None, refract: 0.01, connections: [],
 Node at 234d12621b0 w/ Threshold: 1.0, Leak: None, refract: 0.01, connections: [],
 Node at 234d1263200 w/ Threshold: 1.0, Leak: None, ref

Here's how you can apply a spike to a node:  
```node.apply_spike(amplitude, delay)```
And you can run the network for any amount of time:  
```
net.runfor(t: float)  # runs the network for `t` seconds
net.runtil(t: float)  # runs until the network has simulated `t` seconds total in its lifetime
```

In [107]:

def round_to_nearest(x, div):
    return round(round(x / div) * div, 2)
    
class UnitRateEncoder:
    def __init__(self, nodes, T, refractory=None, amplitude=1):
        self.nodes = nodes
        self.T = T
        self.amplitude = amplitude
        self.allow_zero = True

        if refractory is None:
            self.refractory = self.nodes[0].refractory
            if not isinstance(self.refractory, float):
                raise ValueError("Refractory period not specified and cannot be inferred from nodes")
        else:
            self.refractory = refractory
        
        assert self.nmax > 0
            
    @property
    def nmax(self):
        return int(math.floor(self.T / self.refractory))

    def calculate_nspikes(self, x: float):
        x = max(0, min(1, x))  # constrain to within [0, 1]
        x *= self.nmax + int(self.allow_zero)
        

    def apply_spikes(self, *vector, amplitude=None):
        amplitude = self.amplitude if amplitude is None else amplitude
        # calculate how many spikes
        for node, x in zip(self.nodes, vector):
            for i in range(self.n):
                node.apply_spike(amplitude, self.refractory * i)
        

In [108]:
T = 1.0  # seconds encoding/decoding period
input_encoder = UnitRateEncoder(inputs, T)
output_encoder = UnitRateEncoder(outputs, T)  # for learning signal


In [112]:
inputs[0].apply_spike(1)

In [113]:
net.runfor(1.0)

In [114]:
# Show network output/state
counts = spynns.fires(nodes)
vectors = spynns.vectors(nodes)
lastfires = spynns.lastfires(nodes)
charges = spynns.charges(nodes)
# histories = [node.history for node in nodes]

data = zip(
    range(len(counts)),
    charges,
    counts,
    lastfires,
    vectors,
    # histories,
)
data = [[
    'id',
    'charge',
    'fires',
    't_lastfire',
    't_fires',
    # 'histories',
]] + list(data)
tabulate.tabulate(data, tablefmt='html')

id,charge,fires,t_lastfire,t_fires
0,0,1,1.0,[1.0]
1,0,0,nan,[]
2,0,0,nan,[]
3,0,0,nan,[]
4,0,0,nan,[]
5,0,0,nan,[]
6,0,0,nan,[]
7,1.218289864797625,0,nan,[]
8,0.6497769066717631,0,nan,[]
9,0.03262789228673135,0,nan,[]


In [61]:
net.time  # the network time

9.0

In [62]:
net.queue  # spikes that will get processed in the future

[]

In [63]:
nodes

[Node at 234d12630b0 w/ Threshold: 0.1, Leak: None, refract: 0.01, connections: ['<Edge w: -0.040116884831499666, d: 0.02, dest: 7170>', '<Edge w: -0.12748398390699678, d: 0.02, dest: 72f0>', '<Edge w: -0.3426819530510401, d: 0.02, dest: 6ea0>', '<Edge w: 0.3487215879106287, d: 0.02, dest: 6c90>', '<Edge w: -0.48300642603696087, d: 0.02, dest: 6a80>'],
 Node at 234d1263b60 w/ Threshold: 0.1, Leak: None, refract: 0.01, connections: ['<Edge w: 1.3775553555490212, d: 0.02, dest: 7170>', '<Edge w: 0.8489007725277706, d: 0.02, dest: 72f0>', '<Edge w: 1.6017771451919438, d: 0.02, dest: 6ea0>', '<Edge w: 1.0428847796042242, d: 0.02, dest: 6c90>', '<Edge w: 0.20148188632857145, d: 0.02, dest: 6a80>'],
 Node at 234d12a5610 w/ Threshold: 1.0, Leak: None, refract: 0.01, connections: [],
 Node at 234d12a59d0 w/ Threshold: 1.0, Leak: None, refract: 0.01, connections: [],
 Node at 234d12a5400 w/ Threshold: 1.0, Leak: None, refract: 0.01, connections: [],
 Node at 234d12a6ed0 w/ Threshold: 1.0, Leak:

## Importing/exporting networks
This is not working yet. I based this code off of an importer/exporter I wrote for Tennlab's caspian networks, and I haven't fully made it work here yet.
The code is in the `spynns.network` module in the `network_from_json()` and `to_tennlab()` functions.
`spynns.feedforward` has also not been fully converted over yet either.

Here's how all that stuff worked in my old implementation:

In [ ]:
# import from json file
with open("../experiment_tenn2_tenngineered-milling.json") as f:
    j = json.loads(f.read())

nodes, inputs, outputs = casPYan.network_from_json(j)

In [ ]:
# export to json
d: dict = casPYan.to_tennlab(pnodes, inputs, outputs)
j = json.dumps(d)

## Plasticity/Backprop???

I'm still theorycrafting how to do this.